### Reminder:  Clear output before saving for commit.
otherwise merging will suck.


Main project requirements:
  * few plots per question 2 per question 6-8 plots in total
  * heading for each q and summary of findings under q
  * at least 1 linear regression on at least Q4.
  


In [ ]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
import seaborn as sns 

In [ ]:
file_path = 'Data/database.csv'
df = pd.read_csv(file_path, low_memory=False)  #low memory imput to avoid warning message
# df.info()
# column notes:
# Species Quantity values: '1', '2-10', nan, '11-100', 'Over 100' 

In [ ]:
subset_df = df[["Incident Year","Incident Month","Airport","Airport ID", "State","Aircraft","Aircraft Type", "Flight Phase", "Species Name","Species ID","Species Quantity"]]
subset_df.to_csv("Data/data_subset.csv")


# Which bird is the most dangerous 1990-2014?

In [ ]:
#removing "unknown" bird species, removing partial year data for 2015
birdType_df = subset_df[["Species Name","Species ID","Species Quantity"]].loc[(subset_df["Species Name"]!= "UNKNOWN MEDIUM BIRD") & (subset_df["Species Name"]!= "UNKNOWN SMALL BIRD") 
                                                                              & (subset_df["Species Name"]!= "UNKNOWN LARGE BIRD") & (subset_df["Species Name"]!= "UNKNOWN BIRD")
                                                                              & (subset_df["Incident Year"]!= 2015)]


# todo add labels, pretty up.  add thoughts on results
birdCounts = birdType_df["Species Name"].value_counts()

y_axis = birdCounts[:10]  
x_axis = birdCounts.index[:10]

plt.figure(figsize=(10,5))
plt.bar(x_axis, y_axis, color = "green")
plt.tight_layout()  
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations,birdCounts.index[:10], rotation="vertical")

plt.xlabel('Bird Name') # Label the x axis 
plt.ylabel("Number of Bird Strikes ")  # Label the y axis
plt.title(" Which bird is the most dangerous 1990-2014? aka Mourning Dove ")

for i in range(len(y_axis)):
    plt.annotate(str(y_axis[i]), xy=(x_axis[i],y_axis[i]), ha='center', va='bottom')

plt.savefig("Images/DangerousMourningDove.png") # TODO The legend and axis names are put in the saved image.


plt.show()


#### Which airport or geographic area is more likely to have a bird/plane strike?
    * get fancy add google figure w/heat map possibly
    * Graph for states, or stacked one of all airports grouped by state


In [ ]:
subset_df

In [ ]:
# Jag's \/\/\/
subset_df["State"].value_counts().head(10)

# Get the numnber of occurances of each state
state_counts = df.State.value_counts()

# Split and Save the state names in a variable
states = state_counts.index

# Split and Save the counts in another variable
counts = state_counts.values

# Create barplot object
barplot = sns.barplot(x=states, y=counts)

sns.set(rc={'figure.figsize':(15,6)})
barplot = sns.barplot(x=states[:5], y=counts[:5])


In [ ]:
# Jag's ^^^




# todo add labels, pretty up.  add thoughts on results
stateCounts = subset_df["State"].value_counts()
stateCounts

y_axis = stateCounts[:5]  
x_axis = stateCounts.index[:5]

plt.figure(figsize=(10,5))
plt.bar(x_axis, y_axis, color = "pink")
plt.tight_layout()  
plt.xlabel('State Name')
plt.ylabel("Number of Bird Strikes")
 

for i in range(len(y_axis)):
    plt.annotate(str(y_axis[i]), xy=(x_axis[i],y_axis[i]), ha='center', va='bottom')


plt.show()


In [ ]:
# Selom \/\/\/ - Bird strikes that are increasing or decreasing over time? - 




In [ ]:
#TODO need to clean up this file's first cells and delete the duplicate code from file 1 that builds subset df and just read in the subset df
# Alyssa  \/\/\/ Which phase of the flight reports the most bird strikes?



In [ ]:
#create animal kingdom column in dataframe and populate it for birds/mammals/reptiles involved in airplane incidents.
subset_wkingdom_df = subset_df
#most things in the list are birds, set everything to bird intially
subset_wkingdom_df["Animal_Kingdom"]= "BIRD"
#mammals have species id that starts w/1
subset_wkingdom_df["Animal_Kingdom"]= np.where(subset_wkingdom_df['Species ID'].str.startswith('1',na=False), "MAMMAL",subset_wkingdom_df["Animal_Kingdom"])
#repiles start with 2
subset_wkingdom_df["Animal_Kingdom"]= np.where(subset_wkingdom_df['Species ID'].str.startswith('2',na=False), "REPTILE",subset_wkingdom_df["Animal_Kingdom"])
subset_wkingdom_df.to_csv("Data/data_subset_wKingdom.csv")


In [ ]:
#checkout breakdown as %s
subset_wkingdom_df["Animal_Kingdom"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

## Which phase of the flight reports the most animal strikes?
todo add a bunch of stuff to discuss this q here

Vocab: 
Approach – The phase of flight when the pilot intends to land on the runway. There are different types of approaches, depending on whether the pilot is flying VFR or IFR.

Climb – The act of increasing aircraft altitude, typically to a designated level.

Descent – The act of decreasing aircraft altitude, typically to a designated level.

en Route – In air traveling to a destination.

landing roll- the movement of an aircraft from touchdown through deceleration to taxi speed or full stop.

takeoff run - The distance traveled by an airplane from the commencement of takeoff to the point when it leaves the ground or water.
https://epicflightacademy.com/aviation-terminology/

In [ ]:
subset_df["Flight Phase"].value_counts()
flightphase= subset_df.groupby(['Flight Phase'])
flightphase = flightphase['Incident Year'].count().reset_index().rename(columns={"Incident Year":"Count"})
flightphase = flightphase.sort_values(by='Count',ascending=False).reset_index(drop = True)
#print(flightphase.Count.sum())  check count of sum == the total of non-NaN rows for flight phase 118802
print("Total rows that contain Flight Phase data: "+str(df["Flight Phase"].notnull().sum()))  #118802 rows have flight phase data
#len(df[df["Flight Phase"].isna()])+df["Flight Phase"].notnull().sum() #55302 rows have no flight phase + 118802 = 174104 total rows from orig dataset

#the bottom 6 parts of flight make up only ~1% of the total, so pulled those together to new category "Other"
others_label=flightphase["Flight Phase"][6:]
others_total=flightphase.Count[6:].sum()
new_row = {'Flight Phase':'OTHER', 'Count':others_total}
#append row to the dataframe
flightphase = flightphase.append(new_row, ignore_index=True)
#reorder to move "other" row up to index 6
flightphase = flightphase.sort_values(by='Count',ascending=False).reset_index(drop = True)


In [ ]:
#figure out how to make others label into simple text list  TODO
print(list(others_label))
olist=list(others_label)
#for label in others_label:
#    print(label)
#join makes things a string, between every item, tell it what to put inbetween before the join
print(f"Other category includes the following: {', '.join(olist)}")

In [ ]:
#plot data
pieLabels = flightphase["Flight Phase"][0:7]
#todo get the "count" out of there.
pies = flightphase[0:7].plot.pie(y="Count",figsize=[10, 8],labels=pieLabels,autopct='%1.1f%%',startangle=10)
pies.set_aspect('equal')
pies.legend(loc='center left', bbox_to_anchor=(.95, .75)) #move legend off the pie
plt.title("What phase of flight sustains the most animal strikes")
#todo maybe compare this against injuries/fatalities?

In [ ]:
#plotting by animal kingdom not so iteresting since it's only 3% other
#subset_wkingdom_df.loc[subset_wkingdom_df["Flight Phase"].isin(["APPROACH","TAKEOFF RUN","LANDING ROLL","CLIMB"])].groupby(['Flight Phase', 'Animal_Kingdom']).size().unstack().plot(kind='bar', stacked=True, figsize=(15, 5))
#subset_wkingdom_df.groupby(['Flight Phase', 'Animal_Kingdom']).size().unstack().plot(kind='bar', stacked=True, figsize=(15, 5))
#df.info()
df["Flight Phase"].value_counts()
print(df.shape)  #
df[df["Flight Phase"].isna()]  #55302 rows have no flight phase?


In [ ]:
#todo make another chart to show phase of flight vs. injury/fatality
#drop the rows that don't have injury or fatality

injuryFatality_df = df[['Flight Phase','Injuries', 'Fatalities','Record ID']]  #narrow the column list
injuryFatality_df = injuryFatality_df[injuryFatality_df['Flight Phase'].notna()]  #drop rows w/NaN flight phase 118802 remaining
#drop rows where both injury and fatality are NaN.(ie keep rows that have either a nonna injury or non-na fatality)
injuryFatality_df = injuryFatality_df[(injuryFatality_df['Injuries'].notna()) | (injuryFatality_df['Fatalities'].notna()) ]
#another way to do the same thing below:
#injuryFatality_df =injuryFatality_df.dropna(axis =0, subset=['Injuries', 'Fatalities'], how='all') #drop if both injuries and fatalities are nan
print(injuryFatality_df.shape) #787 rows the subset way
injuryFatality_df=injuryFatality_df.fillna(0) #fill remaining nas w/0s

#keep only rows that have fatality or injury > 0
injuryFatality_df = injuryFatality_df[(injuryFatality_df['Injuries'] > 0) | (injuryFatality_df['Fatalities'] > 0) ]
print(injuryFatality_df.shape)  #down to 237 rows

#injuryFatality_df[['Flight Phase','Injuries', 'Fatalities']].sort_values(by="Fatalities", ascending=False)
inj_fat_gb=injuryFatality_df.groupby(['Flight Phase','Injuries', 'Fatalities']).count().rename(columns={"Record ID":"Count"})
inj_fat_gb#.plot(kind="bar")  #todo  graph this some how

## Where on the aircraft most likely to get damages during a strike? 
somethingsomething 
Figure out how likely animal strike is to result in damage

In [ ]:
damage_subset_df = df[["Airport ID","Species Quantity","Fatalities","Injuries","Aircraft Damage","Radome Strike","Radome Damage","Windshield Strike","Windshield Damage","Nose Strike","Nose Damage","Engine1 Strike","Engine1 Damage","Engine2 Strike","Engine2 Damage","Engine3 Strike","Engine3 Damage","Engine4 Strike","Engine4 Damage","Engine Ingested","Propeller Strike","Propeller Damage","Wing or Rotor Strike","Wing or Rotor Damage","Fuselage Strike","Fuselage Damage","Landing Gear Strike","Landing Gear Damage","Tail Strike","Tail Damage","Lights Strike","Lights Damage","Other Strike","Other Damage"]]


In [ ]:
#futzing cell
print(damage_subset_df["Fatalities"].unique())
print(damage_subset_df["Injuries"].unique())
damage_subset_df["Aircraft Damage"].value_counts().to_frame()
damage_subset_df["Fatalities"].value_counts().to_frame()
df.loc[df["Fatalities"]==5.0]

In [ ]:
stk_dmg_totals =damage_subset_df[["Radome Strike","Radome Damage","Windshield Strike","Windshield Damage","Nose Strike","Nose Damage","Engine1 Strike","Engine1 Damage","Engine2 Strike","Engine2 Damage","Engine3 Strike","Engine3 Damage","Engine4 Strike","Engine4 Damage","Engine Ingested","Propeller Strike","Propeller Damage","Wing or Rotor Strike","Wing or Rotor Damage","Fuselage Strike","Fuselage Damage","Landing Gear Strike","Landing Gear Damage","Tail Strike","Tail Damage","Lights Strike","Lights Damage","Other Strike","Other Damage"]].sum()
stk_dmg_totals = stk_dmg_totals.to_frame()
stk_dmg_totals.rename(columns={0:"Count"}, inplace =True)  #rename column

In [ ]:
orderd_stk_dmg=stk_dmg_totals.sort_values("Count",ascending=False)


#graph the top 10 places to be struck TODO make pretty
# graph the top 10 places to receive damage  todo make pretty
# figure out % of strikes resulting in damage. graph this
#TODO figure out % of strikes resuling in injuries or fatalities

#plot the top 10 places on a the plan struck by animals
orderd_stk_dmg[0:10].plot(kind="bar", color="green")
plt.xlabel('Where on the Plane') # Label the x axis 
plt.ylabel("Number of Bird Strikes ")  # Label the y axis
plt.title("Where on the plane sustains the most animal strikes? (top 10 graphed)")

In [ ]:
dmg_only = stk_dmg_totals.loc[stk_dmg_totals.index.str.contains('Damage')].sort_values("Count",ascending=False)

#plot the top 10 places that reported damage from the animal strike
dmg_only[0:10].plot(kind="bar", color="orange")
plt.xlabel('Where on the Plane') # Label the x axis 
plt.ylabel("Number of Bird Strikes ")  # Label the y axis
plt.title("Where on the plane sustains damamge from animal strikes? (top 10 graphed)")

In [ ]:
damage_pct=orderd_stk_dmg  #made a copy, but it's just a new name for smae thing and "both" are getting updated

In [ ]:
# calculate the % of strikes that caused damage for the top 10 parts of plane to be hit
damage_pct["StrikeDamage%"] = 0.00
damage_pct.at['Radome Strike','StrikeDamage%'] = damage_pct.at['Radome Damage','Count']/damage_pct.at['Radome Strike','Count']*100
damage_pct.at['Windshield Strike','StrikeDamage%'] = damage_pct.at['Windshield Damage','Count']/damage_pct.at['Windshield Strike','Count']*100
damage_pct.at['Nose Strike','StrikeDamage%'] = damage_pct.at['Nose Damage','Count']/damage_pct.at['Nose Strike','Count']*100
damage_pct.at['Wing or Rotor Strike','StrikeDamage%'] = damage_pct.at['Wing or Rotor Damage','Count']/damage_pct.at['Wing or Rotor Strike','Count']*100
damage_pct.at['Fuselage Strike','StrikeDamage%'] = damage_pct.at['Fuselage Damage','Count']/damage_pct.at['Fuselage Strike','Count']*100
damage_pct.at['Other Strike','StrikeDamage%'] = damage_pct.at['Other Damage','Count']/damage_pct.at['Other Strike','Count']*100
damage_pct.at['Engine1 Strike','StrikeDamage%'] = damage_pct.at['Engine1 Damage','Count']/damage_pct.at['Engine1 Strike','Count']*100
damage_pct.at['Landing Gear Strike','StrikeDamage%'] = damage_pct.at['Landing Gear Damage','Count']/damage_pct.at['Landing Gear Strike','Count']*100
damage_pct.at['Engine2 Strike','StrikeDamage%'] = damage_pct.at['Engine2 Damage','Count']/damage_pct.at['Engine2 Strike','Count']*100
damage_pct.at['Lights Strike','StrikeDamage%'] = damage_pct.at['Lights Damage','Count']/damage_pct.at['Lights Strike','Count']*100
damage_pct.sort_values("StrikeDamage%",ascending=False, inplace=True)  #order the df by % just calculated

In [ ]:
overallDamage_v_strikes = damage_subset_df["Aircraft Damage"].value_counts().to_frame()
pct = round(overallDamage_v_strikes.loc[1]/overallDamage_v_strikes.loc[0]*100,2)
print(f"Overall {pct[0]}% Animal strikes resulted in damage to the aircraft." )
#plot percent of strikes that resulted in damage
damage_pct["StrikeDamage%"][0:10].plot(kind="bar",color="red")
#damage_pct_orderd#= damage_pct.sort_values("StrikeDamage%",ascending=False)
#damage_pct_orderd
plt.xlabel('Where on the Plane') # Label the x axis 
plt.ylabel("% of Strikes to result in damage")  # Label the y axis
plt.title("What Percent of strikes result in damage to the plane? (top 10 graphed)")

In [ ]:
# Alyssa ^^^
subset_wkingdom_df.groupby(['Flight Phase', 'Animal_Kingdom'])
subset_wkingdom_df.loc[subset_df["Species ID"].str.startswith('1',na=False)]

#subset_df.loc[subset_df["Species Name"] == "Moose"]